# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [111]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag, ne_chunk
from nltk.corpus import wordnet

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [88]:
# load data from database
DB_NAME = 'disaster.db'
MESSAGES_TABLE = 'messages'

engine = create_engine('sqlite:///{}'.format(DB_NAME))
df = pd.read_sql_table(MESSAGES_TABLE, engine)
print(df.shape)
df.head()

(26216, 40)


,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
child_alone                   0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
related                       0
request                       0
search_a

In [90]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [91]:
# Let's keep only the English version of the messages, as the final classifier
# is based only in English messages
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


In [92]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [93]:
y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [94]:
text = X[3]
text

'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'

In [95]:
def get_wordnet_pos(treebank_tag):
    """ 
    Transforms from Treebank tags to wordnet tags.
    As discussed here: 
    https://stackoverflow.com/questions/15586721/
    wordnet-lemmatization-and-pos-tagging-in-python
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # If unknown, return the default value

In [96]:
# Case normalization
temp_text = text.lower()

# Punctuation removal
temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

# Tokenize
tokens = word_tokenize(temp_text)

# Stop Word Removal
stop_words = stopwords.words("english")
tokens = [word for word in tokens if word not in stop_words]

# Part-of-Speech Tagging
tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

# Named Entity Recognition
# TODO: Add this to the pipeline. The punctuation is important to recognize the
# entities.

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(*token) for token in tokens]

# Stemming
stemmer = PorterStemmer()
tokens = [stemmer.stem(word) for word in tokens]

In [97]:
tokens

['un',
 'report',
 'leogan',
 '80',
 '90',
 'destroy',
 'hospit',
 'st',
 'croix',
 'function',
 'need',
 'suppli',
 'desper']

In [98]:
def tokenize(text):
    """ Basic tokenization function. """
    # Case normalization
    temp_text = text.lower()

    # Punctuation removal
    temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

    # Tokenize
    tokens = word_tokenize(temp_text)

    # Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = [word for word in tokens if word not in stop_words]

    # Part-of-Speech Tagging
    tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

    # Named Entity Recognition
    # TODO: Add this to the pipeline. The punctuation is important to recognize the
    # entities.

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(*token) for token in tokens]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens

In [99]:
%time tokenize(X[0])

CPU times: user 7.71 ms, sys: 8.63 ms, total: 16.3 ms
Wall time: 17.5 ms


['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [100]:
def time_tokenize(X):
    result = list()
    for text in X[:10]:
        result.append(tokenize(text))
    return result

In [101]:
%timeit time_tokenize(X)

16.7 ms ± 852 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [102]:
'Estimated tokenization time: {} seconds'.format((14.6 * 26200 / 10) / 1000)

'Estimated tokenization time: 38.252 seconds'

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [103]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [104]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1cb9f74268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, v

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
%time pipeline.fit(X_train, y_train)

CPU times: user 1min 32s, sys: 5.04 s, total: 1min 37s
Wall time: 1min 39s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [110]:
# What if I use all the cores for the classification part?
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])
%time pipeline.fit(X_train, y_train)

CPU times: user 1min 56s, sys: 3.74 s, total: 2min
Wall time: 1min 16s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

A small improvement (the preprocessing part takes a lot of time).

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [112]:
%time y_pred = pipeline.predict(X_test)

CPU times: user 28.3 s, sys: 1.58 s, total: 29.8 s
Wall time: 28.8 s


In [116]:
y_pred.shape

(7865, 36)

In [120]:
for i in range(y_pred.shape[1]):
    print(classification_report(y_test.values[:, i], y_pred[:, i]))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7769
          1       0.00      0.00      0.00        96

avg / total       0.98      0.99      0.98      7865

             precision    recall  f1-score   support

          0       0.74      0.87      0.80      4539
          1       0.76      0.58      0.66      3326

avg / total       0.75      0.75      0.74      7865

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      7439
          1       0.75      0.11      0.19       426

avg / total       0.94      0.95      0.93      7865

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7865

avg / total       1.00      1.00      1.00      7865

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7742
          1       0.67      0.08      0.14       123

avg / total       0.98      0.98

/Users/antelinvestigacionydesarrollo/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Let's try to get a global score

In [129]:
y_pred.reshape(1, -1).shape

(1, 283140)

In [134]:
y_test.values.reshape(1, -1).shape

(1, 283140)

In [136]:
y_test.values.reshape(1, -1)[0].shape

(283140,)

In [137]:
from sklearn.metrics import f1_score

f1_score(y_test.values.reshape(1, -1)[0], y_pred.reshape(1, -1)[0], average='micro')

0.9424630924630925

What if the labels are takens as vectors?

In [151]:
(y_test.values == y_pred).all(axis=1).mean()

0.2404322949777495

### 6. Improve your model
Use grid search to find better parameters. 

In [155]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1cb9f74268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_state=None, 

In [163]:
parameters = {
    # 'features__text_pipeline__vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__smooth_idf': [True, False],
    'clf__estimator__max_depth': [None, 7]
    # 'clf__estimator__n_estimators': [10, 100],
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)

In [164]:
cv.get_params()

{'cv': None,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1cb9f74268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_

In [165]:
%time cv.fit(X_train, y_train)

CPU times: user 1min 57s, sys: 4.79 s, total: 2min 2s
Wall time: 11min 44s


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__max_depth': [None, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [167]:
best_estimator = cv.best_estimator_
best_estimator.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1cb9f74268>, vocabulary=None)),
  ('tfidf', TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
            use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
               oob_score=False, random_st

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.